In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import time
import threading
import random

# --- नेपाली टाइपिङका लागि अनुच्छेदहरू ---
NEPALI_TEXTS = [
    "नेपाल हिमाल, पहाड र तराई मिलेर बनेको सुन्दर देश हो। यहाँ विभिन्न जातजाति र भाषाभाषीका मानिसहरू मिलेर बसेका छन्।",
    "काठमाडौं नेपालको राजधानी हो। यहाँ पशुपतिनाथ, स्वयम्भूनाथ र बौद्धनाथ जस्ता विश्व सम्पदा सूचीमा सूचीकृत स्थलहरू छन्।",
    "सगरमाथा विश्वको सर्वोच्च शिखर हो, जुन नेपालमा पर्दछ। यसले हरेक वर्ष हजारौं पर्यटकहरूलाई आकर्षित गर्दछ।",
    "नेपाली खाना दाल, भात, तरकारी र अचारको लागि प्रसिद्ध छ। यो स्वादिलो र पौष्टिक हुन्छ। हामीले आफ्नो संस्कृतिलाई माया गर्नुपर्छ।"
]

# --- किबोर्डको लेआउट (React कोडबाट निकालिएको) ---
# प्रत्येक कुञ्जीमा: [primary_char, secondary_char, ref_char]
KEY_LAYOUT = {
    'row1': [['ञ', '॥', '`'], ['१', 'ज्ञ', '1'], ['२', 'ई', '2'], ['३', 'घ', '3'], ['४', 'द्ध', '4'], ['५', 'छ', '5'], ['६', 'ट', '6'], ['७', 'ठ', '7'], ['८', 'ड', '8'], ['९', 'ढ', '9'], ['०', 'ण', '0'], ['औ', 'ओ', '-'], ['+', '=', '=']],
    'row2': [['त्र', 'त्त', 'q'], ['ध', 'ड्ढ', 'w'], ['भ', 'ऐ', 'e'], ['च', 'द्ब', 'r'], ['त', 'ट्ट', 't'], ['थ', 'ठ्ठ', 'y'], ['ग', 'ऊ', 'u'], ['ष', 'क्ष', 'i'], ['य', 'इ', 'o'], ['उ', 'ए', 'p'], ['्र', 'ृ', '['], ['े', 'ै', ']'], ['्', 'ं', '\\']],
    'row3': [['ब', 'आ', 'a'], ['क', 'ङ्क', 's'], ['म', 'ङ्ग', 'd'], ['ा', 'ँ', 'f'], ['न', 'द्द', 'g'], ['ज', 'झ', 'h'], ['व', 'ो', 'j'], ['प', 'फ', 'k'], ['ि', 'ी', 'l'], ['स', 'ट्ठ', ';'], ['ु', 'ू', "'"]],
    'row4': [['श', 'क्क', 'z'], ['ह', 'ह्य', 'x'], ['अ', 'ऋ', 'c'], ['ख', 'ॐ', 'v'], ['द', 'ौ', 'b'], ['ल', 'द्य', 'n'], ['ः', 'ड्ड', 'm'], ['ऽ', 'ङ', ','], ['।', 'श्र', '.'], ['र', 'रु', '/']],
    'row5': [[' ', ' ', ' ']]
}

# --- एप्लिकेसनको अवस्था (State) व्यवस्थापन ---
state = {
    "text_to_type": "",
    "timer_duration": 60,
    "time_left": 60,
    "timer_thread": None,
    "started": False,
    "finished": False,
    "start_time": 0,
    "correct_chars": 0,
    "total_typed_chars": 0,
}

# --- विजेटहरू (UI Components) ---
text_display = widgets.HTML(value="<p style='font-size: 22px; font-family: Preeti, Arial, sans-serif; line-height: 1.8;'>तयार हुनुहोस्...</p>")
stats_display = widgets.HTML(value="<div style='display:flex; justify-content: space-around; font-size: 18px;'>"
                                   "<div>गति: <strong>0 WPM</strong></div>"
                                   "<div>समय: <strong>60</strong> सेकेन्ड</div>"
                                   "<div>शुद्धता: <strong>100%</strong></div></div>")
keyboard_display = widgets.HTML(value="")
user_input = widgets.Textarea(placeholder="टाइप गर्न यहाँ क्लिक गर्नुहोस्...", layout={'width': '99%', 'height': '120px'})
restart_button = widgets.Button(description="पुनः सुरु गर्नुहोस्", icon='redo', button_style='info')

# --- मुख्य कार्यहरू (Core Functions) ---

def generate_keyboard_html(next_char=''):
    """भर्चुअल किबोर्डको HTML बनाउँछ र अर्को कुञ्जीलाई हाइलाइट गर्छ।"""
    html = "<div class='keyboard-container'>"
    key_found = False
    for row_name, keys in KEY_LAYOUT.items():
        html += f"<div class='keyboard-row {row_name}'>"
        for key_data in keys:
            primary, secondary, ref = key_data
            active_class = ""
            if not key_found and (primary == next_char or secondary == next_char or ref == next_char):
                active_class = "active"
                key_found = True

            if ref == ' ':
                html += f"<div class='key space {active_class}'><span>SPACE</span></div>"
            else:
                html += f"<div class='key {active_class}'>" \
                        f"<span class='secondary'>{secondary}</span>" \
                        f"<span class='primary'>{primary}</span>" \
                        f"</div>"
        html += "</div>"
    html += "</div>"
    return html

def update_ui(change=None):
    """प्रयोगकर्ताको इनपुटको आधारमा UI अपडेट गर्छ।"""
    if state['finished']:
        return

    typed_text = user_input.value
    state['total_typed_chars'] = len(typed_text)
    
    # पहिलो पटक टाइप गर्दा टाइमर सुरु गर्ने
    if not state['started'] and state['total_typed_chars'] > 0:
        start_timer()

    # प्रदर्शनको लागि HTML बनाउने
    display_html = ""
    correct_count = 0
    for i, char in enumerate(state['text_to_type']):
        if i < len(typed_text):
            if typed_text[i] == char:
                display_html += f"<span style='color: green;'>{char}</span>"
                correct_count += 1
            else:
                display_html += f"<span style='background-color: #ffcccb;'>{char}</span>"
        elif i == len(typed_text):
            display_html += f"<span style='background-color: lightblue; text-decoration: underline;'>{char}</span>"
        else:
            display_html += f"<span style='color: grey;'>{char}</span>"
    
    state['correct_chars'] = correct_count
    text_display.value = f"<p style='font-size: 22px; font-family: Preeti, Arial, sans-serif; line-height: 1.8;'>{display_html}</p>"

    # तथ्याङ्क गणना र प्रदर्शन
    if state['started']:
        elapsed_time = time.time() - state['start_time']
        wpm = (state['correct_chars'] / 5) / (elapsed_time / 60) if elapsed_time > 0 else 0
        accuracy = (state['correct_chars'] / state['total_typed_chars']) * 100 if state['total_typed_chars'] > 0 else 100
        
        stats_display.value = (f"<div style='display:flex; justify-content: space-around; font-size: 18px;'>"
                               f"<div>गति: <strong>{wpm:.0f} WPM</strong></div>"
                               f"<div>समय: <strong>{state['time_left']}</strong> सेकेन्ड</div>"
                               f"<div>शुद्धता: <strong>{accuracy:.1f}%</strong></div></div>")

    # किबोर्ड अपडेट गर्ने
    next_char = state['text_to_type'][state['total_typed_chars']] if state['total_typed_chars'] < len(state['text_to_type']) else ''
    keyboard_display.value = generate_keyboard_html(next_char)
    
    # पाठ पूरा भएमा
    if state['total_typed_chars'] == len(state['text_to_type']):
        finish_test()

def timer_countdown():
    """टाइमर चलाउने थ्रेड।"""
    while state['time_left'] > 0 and not state['finished']:
        time.sleep(1)
        state['time_left'] -= 1
        # UI is updated from the main input handler to avoid race conditions
    if state['time_left'] == 0:
        finish_test()

def start_timer():
    state['started'] = True
    state['start_time'] = time.time()
    state['timer_thread'] = threading.Thread(target=timer_countdown)
    state['timer_thread'].start()

def finish_test():
    """टाइपिङ टेस्ट समाप्त गर्छ।"""
    if state['finished']: return
    state['finished'] = True
    user_input.disabled = True
    
    elapsed_time = time.time() - state['start_time']
    wpm = (state['correct_chars'] / 5) / (elapsed_time / 60) if elapsed_time > 0 else 0
    accuracy = (state['correct_chars'] / state['total_typed_chars']) * 100 if state['total_typed_chars'] > 0 else 100
    
    text_display.value = (f"<div style='text-align:center; padding: 20px; border: 2px solid green;'>"
                          f"<h2 style='color: green;'>अभ्यास सम्पन्न भयो!</h2>"
                          f"<h3>तपाईंको अन्तिम गति: {wpm:.0f} WPM</h3>"
                          f"<h3>तपाईंको शुद्धता: {accuracy:.1f}%</h3>"
                          f"<p>पुनः अभ्यास गर्न 'पुनः सुरु गर्नुहोस्' बटन थिच्नुहोस्।</p></div>")


def reset_test(button=None):
    """सबै कुरालाई सुरुको अवस्थामा रिसेट गर्छ।"""
    state['finished'] = True # चलिरहेको थ्रेड रोक्न
    time.sleep(0.1)

    state['text_to_type'] = random.choice(NEPALI_TEXTS)
    state['time_left'] = state['timer_duration']
    state['started'] = False
    state['finished'] = False
    state['correct_chars'] = 0
    state['total_typed_chars'] = 0
    
    user_input.value = ""
    user_input.disabled = False
    
    # UI लाई सुरुको अवस्थामा ल्याउने
    initial_char = state['text_to_type'][0]
    keyboard_display.value = generate_keyboard_html(initial_char)
    update_ui()
    
    # focus the textarea for better user experience
    display(HTML("<script>document.querySelector('textarea').focus();</script>"))


# --- Event Handlers and Initial Setup ---
user_input.observe(update_ui, names='value')
restart_button.on_click(reset_test)

# CSS for styling the keyboard and UI
display(HTML("""
<style>
    .keyboard-container { background-color: #f0f0f0; padding: 10px; border-radius: 8px; }
    .keyboard-row { display: flex; justify-content: center; margin-bottom: 5px; }
    .key { 
        display: flex; flex-direction: column; align-items: center; justify-content: center;
        border: 1px solid #ccc; border-radius: 5px; background-color: white;
        width: 50px; height: 50px; margin: 2px; font-size: 16px; font-family: Arial, sans-serif;
        cursor: default; user-select: none; position: relative;
    }
    .key.space { width: 400px; }
    .key .primary { font-size: 1.2em; font-weight: bold; }
    .key .secondary { font-size: 0.8em; color: #888; position: absolute; top: 3px; left: 5px; }
    .key.active { background-color: #007bff; color: white; border-color: #0056b3; }
    .key.active .secondary { color: #eee; }
</style>
"""))

# --- Display the application ---
print("============== नेपाली टाइपिङ अभ्यास ==============")
layout = widgets.VBox([
    stats_display,
    widgets.HTML("<hr>"),
    text_display,
    user_input,
    keyboard_display,
    restart_button
])

reset_test() # पहिलो पटक सुरु गर्न
display(layout)

============== नेपाली टाइपिङ अभ्यास ==============
